In [0]:
from pyspark.sql.functions import *

load_path = "abfss://raw@hisadls.dfs.core.windows.net/Hiring/"
checkpoint_path = "abfss://checkpoint@hisadls.dfs.core.windows.net/hiring_file/"

raw_df = spark.read.option("multiline", "true")\
    .json(load_path)

raw_df.printSchema()

root
 |-- employees: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- profile: struct (nullable = true)
 |    |    |    |-- contact: struct (nullable = true)
 |    |    |    |    |-- address: struct (nullable = true)
 |    |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |    |-- location: struct (nullable = true)
 |    |    |    |    |    |    |-- country: string (nullable = true)
 |    |    |    |    |    |    |-- geo: struct (nullable = true)
 |    |    |    |    |    |    |    |-- lat: string (nullable = true)
 |    |    |    |    |    |    |    |-- long: string (nullable = true)
 |    |    |    |    |    |    |    |-- timezone: struct (nullable = true)
 |    |    |    |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |    |    |    |    |-- utc_offset: string (nullable = true)
 |    |    |    |    

In [0]:
schema = raw_df.schema

bronze_df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "json")\
        .option("multiline", "true")\
        .schema(schema)\
            .load(load_path)

In [0]:
bronze_df.writeStream.format("delta")\
    .option("checkpointLocation", checkpoint_path)\
        .outputMode("append")\
        .trigger(once=True)\
            .table("surakshadb.bronze_stg.hiring_raw")

In [0]:
# spark.read.table("surakshadb.bronze_stg.hiring_raw").display()

employees
